In [4]:
from helper import *
from stats import *
from imp import reload   
import helper
import stats
reload(helper)
reload(stats)
par_get_all_signal

<function helper.par_get_all_signal(signal_name, file_list, product, period, HEAD_PATH='d:/intern', SAVE_PATH='e:/intern', DATA_PATH='d:/intern')>

In [5]:
CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
CORE_NUM

16

- sample product, you can change to your specific product symbol

In [6]:
product_list = ["IF", "IH", "IC", "T", "TF"]

In [7]:
import dask
from dask import compute, delayed

- sample period

In [8]:
period = 4096

In [9]:
all_dates = os.listdir(DATA_PATH + product_list[0])
all_dates.sort()
all_dates = np.array(all_dates)
n_days = len(all_dates)
n_days

1092

- choose an example day

In [23]:
train_sample = all_dates<"2018"

test_sample = all_dates>"2018"


In [24]:
date_str = [n[0:8] for n in all_dates]
format_dates = np.array([pd.to_datetime(d) for d in date_str])
SAVE_PATH = "e:/intern"

- at last we plot the selected pnl

- now we can see that the number of signals are not very big
- and the quality is not so good
- we can propose a systematic method to generate a large number of signals then
- we can choose good ones from them

- for example, we can first generate some signals about volatility without direction
- there are some ideas about it

- standard deviation
- range of price
- ratio of volume to turnover

- then we can generate some signals with direction
- actually all of our previous signals have direction
- but some of them are good and some of them are bad
- we can focus on some good and simple ones, such as

- normalized return
- rsi
- dbook

- let's try to generate range signals first

In [12]:
## calculate rolling standard deviation
from collections import OrderedDict
class foctor_std_period(factor_template):
    factor_name = "std.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(10,13)))
    ])
    
    def formula(self, data, period):
        return np.sqrt(fast_roll_var(data["wpr"], period))

In [13]:
## calcualte rolling variance
def fast_roll_var(x, period):
      x_ma = cum(x,period)/period ## roling first moment
      x2 = x*x
      x2_ma = cum(x2,period)/period ## rolling second moment
      var_x = x2_ma-x_ma*x_ma ## rolling variance
      return(var_x)

In [21]:
x20 = foctor_std_period()
for product in product_list:
    create_signal_path(x20, product, SAVE_PATH);

e:/intern/tmp pkl/IF/std.1024
e:/intern/tmp pkl/IF/std.2048
e:/intern/tmp pkl/IF/std.4096
e:/intern/tmp pkl/IH/std.1024
e:/intern/tmp pkl/IH/std.2048
e:/intern/tmp pkl/IH/std.4096
e:/intern/tmp pkl/IC/std.1024
e:/intern/tmp pkl/IC/std.2048
e:/intern/tmp pkl/IC/std.4096
e:/intern/tmp pkl/T/std.1024
e:/intern/tmp pkl/T/std.2048
e:/intern/tmp pkl/T/std.4096
e:/intern/tmp pkl/TF/std.1024
e:/intern/tmp pkl/TF/std.2048
e:/intern/tmp pkl/TF/std.4096


In [22]:
## rolling price range
from collections import OrderedDict
class foctor_range_period(factor_template):
    factor_name = "range.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(10,13)))
    ])
    
    def formula(self, data, period):
        return data["max."+str(period)]-data["min."+str(period)]

In [23]:
x21 = foctor_range_period()
for product in product_list:
    create_signal_path(x21, product, SAVE_PATH);

e:/intern/tmp pkl/IF/range.1024
e:/intern/tmp pkl/IF/range.2048
e:/intern/tmp pkl/IF/range.4096
e:/intern/tmp pkl/IH/range.1024
e:/intern/tmp pkl/IH/range.2048
e:/intern/tmp pkl/IH/range.4096
e:/intern/tmp pkl/IC/range.1024
e:/intern/tmp pkl/IC/range.2048
e:/intern/tmp pkl/IC/range.4096
e:/intern/tmp pkl/T/range.1024
e:/intern/tmp pkl/T/range.2048
e:/intern/tmp pkl/T/range.4096
e:/intern/tmp pkl/TF/range.1024
e:/intern/tmp pkl/TF/range.2048
e:/intern/tmp pkl/TF/range.4096


In [24]:
## rolling volume/open interest
from collections import OrderedDict
class foctor_volume_open_ratio_period(factor_template):
    factor_name = "volume.open.ratio.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(10,13)))
    ])
    
    def formula(self, data, period):
        return zero_divide(cum(data["qty"],period), cum(data["cum.open.int"]/period,period))

In [25]:
x22 = foctor_volume_open_ratio_period()
for product in product_list:
    create_signal_path(x22, product, SAVE_PATH);


e:/intern/tmp pkl/IF/volume.open.ratio.1024
e:/intern/tmp pkl/IF/volume.open.ratio.2048
e:/intern/tmp pkl/IF/volume.open.ratio.4096
e:/intern/tmp pkl/IH/volume.open.ratio.1024
e:/intern/tmp pkl/IH/volume.open.ratio.2048
e:/intern/tmp pkl/IH/volume.open.ratio.4096
e:/intern/tmp pkl/IC/volume.open.ratio.1024
e:/intern/tmp pkl/IC/volume.open.ratio.2048
e:/intern/tmp pkl/IC/volume.open.ratio.4096
e:/intern/tmp pkl/T/volume.open.ratio.1024
e:/intern/tmp pkl/T/volume.open.ratio.2048
e:/intern/tmp pkl/T/volume.open.ratio.4096
e:/intern/tmp pkl/TF/volume.open.ratio.1024
e:/intern/tmp pkl/TF/volume.open.ratio.2048
e:/intern/tmp pkl/TF/volume.open.ratio.4096


In [26]:
## rolling trend index
from collections import OrderedDict
class foctor_trend_index_period(factor_template):
    factor_name = "trend.index.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(10,13)))
    ])
    
    def formula(self, data, period):
        aa = zero_divide(abs(data["wpr"]-data["wpr"].shift(period)), data["max."+str(period)]-data["min."+str(period)])
        aa[0:period]=0
        return aa

In [27]:
x23 = foctor_trend_index_period()
for product in product_list:
    create_signal_path(x23, product, SAVE_PATH);


e:/intern/tmp pkl/IF/trend.index.1024
e:/intern/tmp pkl/IF/trend.index.2048
e:/intern/tmp pkl/IF/trend.index.4096
e:/intern/tmp pkl/IH/trend.index.1024
e:/intern/tmp pkl/IH/trend.index.2048
e:/intern/tmp pkl/IH/trend.index.4096
e:/intern/tmp pkl/IC/trend.index.1024
e:/intern/tmp pkl/IC/trend.index.2048
e:/intern/tmp pkl/IC/trend.index.4096
e:/intern/tmp pkl/T/trend.index.1024
e:/intern/tmp pkl/T/trend.index.2048
e:/intern/tmp pkl/T/trend.index.4096
e:/intern/tmp pkl/TF/trend.index.1024
e:/intern/tmp pkl/TF/trend.index.2048
e:/intern/tmp pkl/TF/trend.index.4096


- Then we construt these range signals 

In [28]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x20, product=product, HEAD_PATH=SAVE_PATH)
;
# Wall time: 6min 10s


Wall time: 6min 10s


''

In [29]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x21, product=product, HEAD_PATH=SAVE_PATH)
;

#Wall time: 2min 30s

Wall time: 2min 30s


''

In [30]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x22, product=product, HEAD_PATH=SAVE_PATH)
;
# Wall time: 3min 17s



Wall time: 3min 17s


''

In [31]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, os.listdir(DATA_PATH + product)))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x23, product=product, HEAD_PATH=SAVE_PATH)
;

## Wall time: 4min 8s

Wall time: 4min 8s


''

- Now we can combine directional signals and range signals together to become new direction signals

- For example, we have n directional signals, and m range signals, 
- then we can generate new n*m directional signals

In [32]:
## generate composite signal: trend signal * range signal = new trend signal
def construct_composite_signal(dire_signal, range_signal, period_list, date_list, product_list, HEAD_PATH):
    from collections import OrderedDict
    class foctor_xx_period(factor_template):
        factor_name = dire_signal+"."+range_signal+".period" ## name of new signal
        params = OrderedDict([
            ("period", period_list)
        ])
        def formula(self, data, period):
            return (data[dire_signal+"."+str(period)]*data[range_signal+"."+str(period)]).values ## calculation of new signal
    xx = foctor_xx_period()
    for product in product_list:
        create_signal_path(xx, product, HEAD_PATH)
        file_list = [DATA_PATH+product+"/"+date for date in date_list]
        parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=xx, product=product, HEAD_PATH=HEAD_PATH)

In [33]:
dire_signal_list = ["trade.imb","total.trade.imb", "nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["range", "std", "volume.open.ratio", "trend.index"]
period_list = np.power(2, range(10,13))

In [34]:
%%time
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        construct_composite_signal(dire_signal, range_signal, period_list, all_dates, product_list, SAVE_PATH)
##　Wall time: 5h 24min 2s

e:/intern/tmp pkl/IF/trade.imb.range.1024
e:/intern/tmp pkl/IF/trade.imb.range.2048
e:/intern/tmp pkl/IF/trade.imb.range.4096
e:/intern/tmp pkl/IH/trade.imb.range.1024
e:/intern/tmp pkl/IH/trade.imb.range.2048
e:/intern/tmp pkl/IH/trade.imb.range.4096
e:/intern/tmp pkl/IC/trade.imb.range.1024
e:/intern/tmp pkl/IC/trade.imb.range.2048
e:/intern/tmp pkl/IC/trade.imb.range.4096
e:/intern/tmp pkl/T/trade.imb.range.1024
e:/intern/tmp pkl/T/trade.imb.range.2048
e:/intern/tmp pkl/T/trade.imb.range.4096
e:/intern/tmp pkl/TF/trade.imb.range.1024
e:/intern/tmp pkl/TF/trade.imb.range.2048
e:/intern/tmp pkl/TF/trade.imb.range.4096
e:/intern/tmp pkl/IF/total.trade.imb.range.1024
e:/intern/tmp pkl/IF/total.trade.imb.range.2048
e:/intern/tmp pkl/IF/total.trade.imb.range.4096
e:/intern/tmp pkl/IH/total.trade.imb.range.1024
e:/intern/tmp pkl/IH/total.trade.imb.range.2048
e:/intern/tmp pkl/IH/total.trade.imb.range.4096
e:/intern/tmp pkl/IC/total.trade.imb.range.1024
e:/intern/tmp pkl/IC/total.trade.imb.

- Now we can combine signals of each day in to a vector
- here we use 4096 as period
- so to keep them independent, we choose 1 out of every 4096 points

In [25]:
period = 4096
os.makedirs(HEAD_PATH+"/all signal", exist_ok=True)
dire_signal_list = ["trade.imb","total.trade.imb", "nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["", "range", "std", "volume.open.ratio", "trend.index"]
all_signal_list = np.array([])
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        if len(range_signal)==0:
            signal_name = dire_signal
        else:
            signal_name = dire_signal+"."+range_signal
        all_signal_list = np.append(all_signal_list,signal_name)
all_period_signal = [signal+".4096" for signal in all_signal_list]
len(all_period_signal)
## 45

45

- we have 9 directional (trend) signals and 4 range signals
- so we have 36 composite signals 
- plus the 9 original ones we have 45 signals together


In [36]:
## save the good column to separte files
def get_good(date, product, HEAD_PATH, SAVE_PATH):
    data = load(HEAD_PATH+"/pkl tick/"+product+"/"+date)
    good = data["good"]
    save(good, SAVE_PATH+"/good pkl/"+product+"/"+date)  

In [37]:
os.makedirs(SAVE_PATH+"/good pkl", exist_ok=True)
for product in product_list:
    os.makedirs(SAVE_PATH+"/good pkl/"+product, exist_ok=True)

In [38]:
%%time
for product in product_list:
    parLapply(CORE_NUM, all_dates, get_good, product=product, HEAD_PATH=HEAD_PATH, SAVE_PATH="e:/intern")
    
# Wall time: 5min 13s


Wall time: 5min 13s


In [39]:
def moving_average(a, n=3) :
    ret_sum = np.cumsum(a, dtype=float)
    ret = a
    ret[n:] = (ret_sum[n:] - ret_sum[:-n])/n
    return ret

In [40]:
## parallel generate the distribution of a signal
def par_get_all_signal(signal_name, file_list, product, period, HEAD_PATH="d:/intern", SAVE_PATH="e:/intern", DATA_PATH="d:/intern"):
    n_files = len(file_list)
    all_signal = np.array([])
    for file in file_list:
        S = load(HEAD_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file) ## signal
        good = load(DATA_PATH+"/good pkl/"+product+"/"+file) ## good singal
        signal = S[good]
        moving_average(signal,period)
        chosen = (np.arange(len(signal))+1) % period==0
        all_signal = np.concatenate((all_signal, signal[chosen]), axis=0)
    save(all_signal, SAVE_PATH+"/all signal/"+product+"."+signal_name+".pkl")

In [41]:
%%time
for product in product_list:
    parLapply(CORE_NUM, all_period_signal, par_get_all_signal, file_list=all_dates, product=product, period=4096, HEAD_PATH=SAVE_PATH,
             SAVE_PATH = "e:/intern", DATA_PATH="e:/intern");
## Wall time: 1h 16min 9s


Wall time: 1h 16min 9s


- then we can backtest our signals 
- we save the results

In [42]:
## this is to evalute the performance of a signal on a product
def evaluate_signal(signal, all_dates, product, min_pnl, min_num, 
                    CORE_NUM, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern", period=4096, split_str="2018", tranct=1.1e-4, 
                    max_spread=0.61, tranct_ratio=True, atr_filter=0, save_path="signal result",reverse=0):
    signal_name = signal+"."+str(period) ## signal name, with period
    all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+signal_name+".pkl") ## get the distribution of the signal
    tranct = product_info[product]["tranct"] ## transaction cost of the product
    tranct_ratio = product_info[product]["tranct.ratio"] ## True: based on notional; False: fixed tranct
    open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001))) ## open threshold
    thre_list = []
    for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])): ## close threshold
        thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
    thre_list = np.array(thre_list)
    thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])) ## threshold matrix
    if reverse>=0: ## trending signal
        print("reverse=1")
        trend_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str="2018", reverse=1, tranct=tranct, 
                                   max_spread=max_spread, tranct_ratio=tranct_ratio, min_pnl=min_pnl, min_num=min_num, atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH)
    if reverse<=0: ## reversal signal
        print("reverse=-1")
        reverse_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str="2018", reverse=-1, tranct=tranct, 
                        max_spread=max_spread, tranct_ratio=tranct_ratio, min_pnl=min_pnl, min_num=min_num, atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH)
    if reverse==0: ## both trending and reversal
        stat_result = OrderedDict([("trend.signal.stat", trend_signal_stat), ("reverse.signal.stat", reverse_signal_stat)])    
        save(stat_result, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".pkl")
    elif reverse==1: ## just trend
        save(trend_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".trend.pkl")
    elif reverse==-1: ## just reversal
        save(reverse_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".reverse.pkl")

In [43]:
## get the statistics of backtesting a signal
from collections import OrderedDict
def get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str="2018", reverse=1, tranct=1.1e-4, 
                    max_spread=0.61, tranct_ratio=True, min_pnl=2, min_num=20, atr_filter=0, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern"):
    train_sample = all_dates<split_str ## training samples
    test_sample = all_dates>split_str ## testing samples
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=reverse, tranct=tranct, max_spread=max_spread, tranct_ratio=tranct_ratio, 
                                  HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH, atr_filter=atr_filter)
        train_result = compute([delayed(f_par)(file) for file in all_dates[train_sample]])[0] ## get training result
    train_stat = get_hft_summary(train_result, thre_mat, sum(train_sample)) ## get training result statistics
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                  reverse=reverse, tranct=tranct, max_spread=max_spread, tranct_ratio=tranct_ratio, 
                                  HEAD_PATH=HEAD_PATH, SIGNAL_PATH=SIGNAL_PATH, atr_filter=atr_filter) 
        test_result = compute([delayed(f_par)(file) for file in all_dates[test_sample]])[0] ## get testing result
    test_stat = get_hft_summary(test_result, thre_mat, sum(test_sample)) ## get testing result statistics
    return OrderedDict([("train.stat", train_stat), ("test.stat", test_stat)])

In [44]:
%%time
SIGNAL_PATH = "e:/intern"
for product in product_list:
    spread = product_info[product]["spread"]
    tranct = product_info[product]["tranct"]
    tranct_ratio = product_info[product]["tranct.ratio"]
    if product=="IF" or product=="IH" or product=="IC":
        atr_filter = 100
        avg_pnl = 10
    else:
        atr_filter = 20
        avg_pnl = 2
    for signal in all_signal_list:
        print(product, signal)
        evaluate_signal(signal, all_dates, product, avg_pnl, 20, CORE_NUM, HEAD_PATH, SIGNAL_PATH,
                period=4096, split_str="2018", tranct=tranct, 
                max_spread=spread*1.1, tranct_ratio=tranct_ratio, atr_filter=atr_filter, save_path="signal result atr")

## Wall time: 8h 36min 2s

IF trade.imb
reverse=1
reverse=-1
IF total.trade.imb
reverse=1
reverse=-1
IF nr
reverse=1
reverse=-1
IF dbook
reverse=1
reverse=-1
IF range.pos
reverse=1
reverse=-1
IF price.osci
reverse=1


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=-1
IF ma.dif.10
reverse=1
reverse=-1
IF kdj.k
reverse=1
reverse=-1
IF kdj.j
reverse=1
reverse=-1
IF trade.imb.range
reverse=1


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=-1
IF total.trade.imb.range
reverse=1


...

reverse=-1
TF kdj.k.trend.index
reverse=1


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=-1
TF kdj.j.trend.index
reverse=1


d:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


reverse=-1
Wall time: 8h 36min 2s


- we can write a function to get all the signals' performance

- we can get the result with atr and without atr separately

In [43]:
## get the signal performance
## including trend and reverse signals
def get_signal_performance_result(all_period_signal, signal_dire, period, product_list, train_sample, test_sample):
    trend_signal_result = pd.DataFrame(data=OrderedDict([("signal", all_period_signal), ("reverse",1),
                               ("num", 0), ("trainSharpe", 0), ("testSharpe", 0)]))
    reverse_signal_result = pd.DataFrame(data=OrderedDict([("signal", all_period_signal), ("reverse",-1),
                               ("num", 0), ("trainSharpe", 0), ("testSharpe", 0)]))
    n_signal = len(all_period_signal) ## number of all signals
    for k in range(n_signal):
        test_all_pnl = np.zeros([sum(test_sample), len(product_list)]) ## test pnl
        train_all_pnl = np.zeros([sum(train_sample), len(product_list)]) ## train pnl
        signal_name = all_period_signal[k]
        i = 0
        for product in product_list:
            if product=="IF" or product=="IH" or product=="IC":
                avg_pnl = 10
            else:
                avg_pnl = 2
            stat_result = load(HEAD_PATH+"/" + signal_dire +"/"+product+"."+signal_name+".pkl") ## statistics of signal over a product
            trend_signal_stat = stat_result['trend.signal.stat'] ## trending statistics
            spread = product_info[product]["spread"]
            if tuple(trend_signal_stat.keys())[0]=='train.stat':
                train_stat = trend_signal_stat["train.stat"]
                test_stat = trend_signal_stat["test.stat"]
                #good_strat = trend_signal_stat["good.strat"]
                good_strat = (train_stat["final.result"]["avg.pnl"]>avg_pnl*spread) & (train_stat["final.result"]["num"]>10) ## filter criterion
                if sum(good_strat)>2:
                    train_stat = trend_signal_stat["train.stat"]
                    test_stat = trend_signal_stat["test.stat"]
                    train_pnl = train_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat) ## get the daily return
                    #train_std = np.std(train_pnl)
                    #train_pnl = train_pnl/train_std
                    test_pnl = test_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat)
                    #print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))
                    test_all_pnl[:,i] = test_pnl
                    train_all_pnl[:,i] = train_pnl
                    i = i+1
            if i>0: ## if there are any good products
                train_portfolio = np.array(np.mean(train_all_pnl[:,0:i], axis=1)) ## training portfolio
                test_portfolio = np.array(np.mean(test_all_pnl[:,0:i], axis=1)) ## testing portfolio
                all_portfolio = np.append(train_portfolio, test_portfolio)
                trend_signal_result.loc[k, ("signal", "num", "trainSharpe", "testSharpe")] = (signal_name, i,  sharpe(train_portfolio),sharpe(test_portfolio))
        test_all_pnl = np.zeros([sum(test_sample), len(product_list)])
        train_all_pnl = np.zeros([sum(train_sample), len(product_list)])
        i = 0
        for product in product_list:
            stat_result = load(HEAD_PATH+"/"+signal_dire+"/"+product+"."+signal_name+".pkl")
            reverse_signal_stat = stat_result['reverse.signal.stat']
            spread = product_info[product]["spread"]
            if tuple(reverse_signal_stat.keys())[0]=='train.stat':
                #good_strat = reverse_signal_stat["good.strat"]
                train_stat = reverse_signal_stat["train.stat"]
                test_stat = reverse_signal_stat["test.stat"]
                good_strat = (train_stat["final.result"]["avg.pnl"]>avg_pnl*spread) & (train_stat["final.result"]["num"]>10)
                if sum(good_strat)>2:
                    train_pnl = train_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat)
                    #train_std = np.std(train_pnl)
                    #train_pnl = train_pnl/train_std
                    test_pnl = test_stat["daily.ret"].loc[:, good_strat].sum(axis=1)/sum(good_strat)
                    test_all_pnl[:,i] = test_pnl
                    train_all_pnl[:,i] = train_pnl
                    i = i+1
            if i>0:
                train_portfolio = np.array(np.mean(train_all_pnl[:,0:i], axis=1))
                test_portfolio = np.array(np.mean(test_all_pnl[:,0:i], axis=1))
                all_portfolio = np.append(train_portfolio, test_portfolio)
                reverse_signal_result.loc[k, ("signal","num", "trainSharpe", "testSharpe")] = (signal_name, i, sharpe(train_portfolio),sharpe(test_portfolio))
    return OrderedDict([("trend.signal.stat", trend_signal_result), 
                        ("reverse.signal.stat", reverse_signal_result)])


In [44]:
result_atr = get_signal_performance_result(all_period_signal, "signal result atr", 4096, product_list, train_sample, test_sample)

In [45]:
def get_single_signal_stat(result_atr, min_num=0, min_sharpe=0.5):
    good_trend = (result_atr["trend.signal.stat"]["num"]>min_num) & (result_atr["trend.signal.stat"]["trainSharpe"]>min_sharpe) & (result_atr["trend.signal.stat"]["testSharpe"]>min_sharpe)
    good_reverse = (result_atr["reverse.signal.stat"]["num"]>min_num) & (result_atr["reverse.signal.stat"]["trainSharpe"]>min_sharpe) & (result_atr["reverse.signal.stat"]["testSharpe"]>min_sharpe)
    print("with atr good signals: "+ str(sum(good_trend | good_reverse)))
    train_sharpe = np.mean(np.append(result_atr["trend.signal.stat"]["trainSharpe"][good_trend],
                      result_atr["reverse.signal.stat"]["trainSharpe"][good_reverse]))
    test_sharpe = np.mean(np.append(result_atr["trend.signal.stat"]["testSharpe"][good_trend],
                      result_atr["reverse.signal.stat"]["testSharpe"][good_reverse]))
    print("train sharpe: ", train_sharpe)
    print("test sharpe: ", test_sharpe)
    print(all_signal_list[good_trend])
    print(all_signal_list[good_reverse])


In [46]:
get_single_signal_stat(result_atr, min_num=0, min_sharpe=0.5)

# with atr good signals: 9
# train sharpe:  0.8992799496158685
# test sharpe:  0.7257340111924915
# ['nr' 'dbook' 'kdj.j' 'total.trade.imb.std' 'nr.std']
# ['total.trade.imb' 'dbook' 'trade.imb.std' 'dbook.std'
#  'kdj.k.volume.open.ratio']





with atr good signals: 9
train sharpe:  0.8992799496158685
test sharpe:  0.7257340111924915
['nr' 'dbook' 'kdj.j' 'total.trade.imb.std' 'nr.std']
['total.trade.imb' 'dbook' 'trade.imb.std' 'dbook.std'
 'kdj.k.volume.open.ratio']


In [47]:
get_single_signal_stat(result_atr, min_num=1, min_sharpe=0.5)

# with atr good signals: 7
# train sharpe:  0.9900414492692252
# test sharpe:  0.7111257048084579
# ['nr' 'dbook' 'kdj.j' 'total.trade.imb.std' 'nr.std']
# ['trade.imb.std' 'kdj.k.volume.open.ratio']




with atr good signals: 7
train sharpe:  0.9900414492692252
test sharpe:  0.7111257048084579
['nr' 'dbook' 'kdj.j' 'total.trade.imb.std' 'nr.std']
['trade.imb.std' 'kdj.k.volume.open.ratio']


In [30]:
get_single_signal_stat(result_atr, min_num=2, min_sharpe=0.5)

# with atr good signals: 5
# train sharpe:  1.1122554570979415
# test sharpe:  0.6958291373016495
# ['nr' 'dbook' 'kdj.j' 'total.trade.imb.std' 'nr.std']
# []



with atr good signals: 5
train sharpe:  1.1122554570979415
test sharpe:  0.6958291373016495
['nr' 'dbook' 'kdj.j' 'total.trade.imb.std' 'nr.std']
[]


In [31]:
get_single_signal_stat(result_atr, min_num=3, min_sharpe=0.5)

# with atr good signals: 3
# train sharpe:  1.3090453422897632
# test sharpe:  0.7333286271985716
# ['kdj.j' 'total.trade.imb.std' 'nr.std']
# []





with atr good signals: 3
train sharpe:  1.3090453422897632
test sharpe:  0.7333286271985716
['kdj.j' 'total.trade.imb.std' 'nr.std']
[]


In [32]:
get_single_signal_stat(result_atr, min_num=4, min_sharpe=0.5)

# with atr good signals: 1
# train sharpe:  1.049611958678093
# test sharpe:  0.6410314101993153
# ['total.trade.imb.std']
# []


with atr good signals: 1
train sharpe:  1.049611958678093
test sharpe:  0.6410314101993153
['total.trade.imb.std']
[]


- we can see that their results are very similar

- Conclusion
- This week we have generated results for all of the 35 signals
- We have compared backtest result with and without atr
- we find that there is not much difference between them
- previously we found that for iorn with atr is better than without atr
- that's because we lower the threhsold for number of trades
- now we use 20 trades for training maybe for i there are not enough trades
- Anyway, currently there are too many signals and hard to tune parameters
- in the future, we can use fewer signals so that we can do the research more efficiently